<h2><b>STEP 1 — Imports (TensorFlow / Keras)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np


In [ ]:
print(tf.__version__)


2.19.0


<h2><b>Load CIFAR-10 dataset (raw)

In [ ]:
# Download CIFAR-10 dataset and print its shape

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

# (sample size, height, width, channels)

(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


<h2><b>Inspect class names (ground truth)

In [ ]:
# print the class index with its name.

cifar10_classes = [
    "airplane", "automobile", "bird", "cat", "deer",
    "dog", "frog", "horse", "ship", "truck"
]

for i, name in enumerate(cifar10_classes):
    print(i, name)


0 airplane
1 automobile
2 bird
3 cat
4 deer
5 dog
6 frog
7 horse
8 ship
9 truck


<h2><b>Create CIFAR-5 subset

In [ ]:
# ['airplane', 'automobile', 'bird', 'cat', 'deer']
selected_ids = [0, 1, 2, 3, 4]

# np.isin(elements, test_elements)
# for every element in elements, is element present in test_elements?
# returns array of bools of length elements

train_mask = np.isin(y_train.flatten(), selected_ids)
test_mask = np.isin(y_test.flatten(), selected_ids)

x_train_5 = x_train[train_mask]
y_train_5 = y_train[train_mask]

x_test_5 = x_test[test_mask]
y_test_5 = y_test[test_mask]

print(x_train_5.shape, y_train_5.shape)
print(x_test_5.shape, y_test_5.shape)


(25000, 32, 32, 3) (25000, 1)
(5000, 32, 32, 3) (5000, 1)


<h2><b>Preprocess for MobileNetV2

MobileNetV2 is a lightweight Convolutional Neural Network (CNN) architecture developed by Google.
Traditional CNNs like:

VGG,
ResNet,
Inception

are:

Very large,
Compute-heavy,
Not ideal for mobile devices, browsers, or low-cost GPUs

ImageNet is a massive, labeled image database for computer vision research, organizing over 14 million high-resolution images into a hierarchy of 20,000+ categories

Images are annotated by human labelers, providing rich data for training.

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Your images look numerically similar to what MobileNetV2 saw during pretraining
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224))
    image = preprocess_input(image)
    return image, label

BATCH_SIZE = 64

train_ds = tf.data.Dataset.from_tensor_slices((x_train_5, y_train_5))
train_ds = train_ds.shuffle(1000) # Randomizes sample order
train_ds = train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.batch(BATCH_SIZE) # (64, 224, 224, 3)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE) # While the GPU trains on batch N, CPU prepares batch N+1

test_ds = tf.data.Dataset.from_tensor_slices((x_test_5, y_test_5))
test_ds = test_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)


<h2><b>Build MobileNetV2 model (Frozen Backbone) — Case 2.a

In [ ]:
# Load the pretrained model MobileNetV2

# Top: The original classifier head (1000 classes). base_model is only the feature extractor part.
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)

# Freeze the feature extractor. We want to reuse the knowledge of the pretrained model.
base_model.trainable = False

In [ ]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(5, activation="softmax")
])

model.summary()

# sequential: linear models
# functional: non linear workflows


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │         6,405 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,264,389 (8.64 MB)

 Trainable params: 6,405 (25.02 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

<h2><b>Compile model

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Optimiser: Adam = adaptive LR.
# Loss = sparse_categorical_crossentropy =

<h2><b> Train Model



In [ ]:
history_2a = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=5
)


Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 43s 90ms/step - accuracy: 0.7974 - loss: 0.5523 - val_accuracy: 0.8972 - val_loss: 0.2833
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 24s 61ms/step - accuracy: 0.9006 - loss: 0.2709 - val_accuracy: 0.9042 - val_loss: 0.2643
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 41s 61ms/step - accuracy: 0.9113 - loss: 0.2425 - val_accuracy: 0.9040 - val_loss: 0.2518
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 41s 61ms/step - accuracy: 0.9167 - loss: 0.2270 - val_accuracy: 0.9042 - val_loss: 0.2640
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 24s 61ms/step - accuracy: 0.9197 - loss: 0.2200 - val_accuracy: 0.9096 - val_loss: 0.2502


In [ ]:
test_loss_2a, test_acc_2a = model.evaluate(test_ds)
print("Test Loss:", test_loss_2a)
print("Test Accuracy:", test_acc_2a)


79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 48ms/step - accuracy: 0.9070 - loss: 0.2582
Test Loss: 0.25023704767227173
Test Accuracy: 0.909600019454956


<h2><b>Case 2.b — Partial Fine-Tuning

In [ ]:
import tensorflow as tf
tf.keras.backend.clear_session()


In [ ]:
# Recreate base model
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)

# Enable fine-tuning
base_model.trainable = True

FINE_TUNE_AT = len(base_model.layers) - 30
for layer in base_model.layers[:FINE_TUNE_AT]:
    layer.trainable = False

In [ ]:

# Rebuild full model
model_2b = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(5, activation="softmax")
])

model_2b.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
model_2b.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │         6,405 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,264,389 (8.64 MB)

 Trainable params: 1,532,805 (5.85 MB)

 Non-trainable params: 731,584 (2.79 MB)

In [ ]:
history_2b = model_2b.fit(
    train_ds,
    validation_data=test_ds,
    epochs=5
)

Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 57s 111ms/step - accuracy: 0.8505 - loss: 0.4109 - val_accuracy: 0.9028 - val_loss: 0.2756
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 29s 75ms/step - accuracy: 0.9638 - loss: 0.1050 - val_accuracy: 0.9050 - val_loss: 0.2857
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 30s 77ms/step - accuracy: 0.9871 - loss: 0.0428 - val_accuracy: 0.9170 - val_loss: 0.2990
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 40s 74ms/step - accuracy: 0.9949 - loss: 0.0196 - val_accuracy: 0.9418 - val_loss: 0.2116
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 41s 75ms/step - accuracy: 0.9969 - loss: 0.0118 - val_accuracy: 0.9366 - val_loss: 0.2358


In [ ]:
test_loss_2b, test_acc_2b = model_2b.evaluate(test_ds)
print("Test Loss:", test_loss_2b)
print("Test Accuracy:", test_acc_2b)


79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - accuracy: 0.9377 - loss: 0.2411
Test Loss: 0.23578721284866333
Test Accuracy: 0.9366000294685364
